In [1]:

# HIGH PRIORITY:

# IMPORTANT
# - Does it run from scratch on a new computer?
# - Add saved buffers/trajs to the real repo
# - change num_rollouts back
# - Add instructions

# Add demos option
# Add preloaded
# Actually collect and check

# STRETCH
# - Setup.py
# - Collect val set
# - Hide unnecessary printing
# - make videos better
# - mf_trainer logging
# - partial observability
# - log more often
# - Add early stopping on training
# - maybe add training error bar?


# By 6:10
# - commit
# - Message Ab
# - hide unnecessary stuff

# - By 7,
# - commit
# - Message Ab
# - Add instructions

# Setup - do this once

In [2]:
!git clone https://github.com/aliengirlliv/teachable 1> /dev/null

fatal: destination path 'teachable' already exists and is not an empty directory.


In [14]:
cd teachable

[Errno 2] No such file or directory: 'teachable'
/home/olivia/Documents/Teachable/demo/teachable


In [15]:
!git checkout sac 1> /dev/null
!pip install -r reqs.txt 1> /dev/null

Already on 'sac'


# Setup - Do this each time you reload the notebook

In [5]:
%matplotlib tk

In [6]:
cd teachable

/home/olivia/Documents/Teachable/demo/teachable


In [7]:
from final_demo import *
import ipywidgets as widgets
from IPython.display import HTML

In [8]:
cd teachable

[Errno 2] No such file or directory: 'teachable'
/home/olivia/Documents/Teachable/demo/teachable


# Selections

(CHOOSE ONCE; RE-RUN IF NEEDED)

In [9]:
widgets.Box(items)

Box(children=(ToggleButtons(description='Environment', options=('Ant', 'BabyAI'), value='Ant'), ToggleButtons(…

In [10]:
env_type = 'BabyAI'
collect_type = 'Advice'
save_path = 'one_more_time'

# Collection

TODO: INSERT DESCRIPTION

In [11]:
collector = HumanFeedback(env_type=env_type, collect_type=collect_type, save_path=save_path)

using seed 1


In [12]:
assert False

AssertionError: 

# Train

Here, we train a policy using the buffer of collected trajectories.


In [ ]:
args = make_args(collector, save_path)  
run_experiment(args)

# Visualize

Play video saved during training

In [ ]:
html_str = display_trained_model(save_path)
HTML(html_str)